Let aussume that we need to make this computation

$\sqrt{|(a+b)\times c|}$

and we want to compute the standard deviation of the result assuming that a, b and c are normal distributed independent variables. Clearly the problem is quite naive but we want to address is as if we will need a cluster to solve it. 

We can partition the problem in a three conscutive operations
1. A sum: $(a+b)$
2. A multiplication of the result 1 with c: $(a+b)\times c$
3. A sqrt of the result of 2: $\sqrt{|(a+b)\times c|}$

See https://gitlab.cern.ch/abpcomputing/sandbox/tree_maker for the installation.

Documentation (only started, you need to be on GPN) can be found at https://acc-py.web.cern.ch/gitlab/abpcomputing/sandbox/tree_maker/docs/master/. 

In [1]:
import tree_maker
from tree_maker import NodeJob

In [2]:
# Clearly for this easy task on can do all in the very same python kernel
# BUT here we want to mimic the typical flow
# 1. MADX for optics matching/error seeding
# 2. Tracking for FMA and or DA studies
# 3. simulation baby-sitting and
# 4. postprocessing

import numpy as np
a=np.random.randn(4)
b=np.random.randn(4)
c=np.random.randn(2)

my_list_original=[]
for ii in c:
    my_list_original+=list(np.sqrt(np.abs((a+b)*ii)))
my_list_original=sorted(my_list_original)

In [3]:
#root
root = NodeJob(name='root', parent=None)
# to be modified accordingly
root.path = '/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000'
root.dictionary = {'log_file': f"{root.path}/log.yaml"}
root.clean_log()

In [4]:
#first generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child:03}",
                           parent=node,
                           path = f"{node.path}/{child:03}",
                           template_path = root.path+'/../templates/sum_it',
                           submit_command = f'python run.py',
                           dictionary={'a':float(a[child]), 
                                       'b':float(b[child]),
                                       'log_file': f"{node.path}/{child:03}/log.yaml"
                                      })
                   for child in range(len(a))]

# To combine different lists one can use the product or the zip functions    
#import itertools
#[[i, j, z] for i, j, z in itertools.product(['a','b'],['c','d'],[1,2,3])]
#[[i, j, z] for i, j, z in zip(['a','b'],['c','d'],[1,2,3])]
root.print_it()

root
├── 000
├── 001
├── 002
╰── 003


In [5]:
#second generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child:03}",
                           parent=node,
                           path = f"{node.path}/{child:03}",
                           template_path = root.path+'/../templates/multiply_it',
                           submit_command = f'python run.py',
                           dictionary={'parent':f'{node.path}',
                                       'c': float(c[child]),
                                       'log_file': f'{node.path}/{child:03}/log.yaml',
                                      })
                   for child in range(len(c))]
root.print_it()

root
├── 000
│   ├── 000
│   ╰── 001
├── 001
│   ├── 000
│   ╰── 001
├── 002
│   ├── 000
│   ╰── 001
╰── 003
    ├── 000
    ╰── 001


In [6]:
#third generation
for node in root.leaves:
    node.children=[NodeJob(name=f"{child:03}",
                           parent=node, 
                           path = f"{node.path}/{child:03}",
                           template_path = root.path+'/../templates/square_root_it',
                           submit_command = f'python run.py',
                           dictionary={'parent':f'{node.path}',
                                       'log_file': f"{node.path}/{child:03}/log.yaml",
                                       'test': {'is an example':4}
                                      })
                           for child in range(1)]
root.print_it()

root
├── 000
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
├── 001
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
├── 002
│   ├── 000
│   │   ╰── 000
│   ╰── 001
│       ╰── 000
╰── 003
    ├── 000
    │   ╰── 000
    ╰── 001
        ╰── 000


In [7]:
# we can also modify the submit command
if False:
for i, node in enumerate(root.leaves):
    if i>3:
        print(i)
        node.submit_command = f'condor_submit run.sub -batch-name square_root'

In [8]:
# we can inspect the data structure
root.children[3].children[1].children[0].submit_command

'python run.py'

In [9]:
root.to_yaml()

In [10]:
# important concept of generation
root.generation(2)

In [25]:
# We map the pythonic tree in a >folder< tree

root.clean_log()
root.rm_children_folders()
for depth in range(root.height):
    [x.clone_children() for x in root.generation(depth)]

root.tag_as('cloned')

In [12]:
root.tag_as('launched')
for node in root.generation(1):
    node.clean_log()
    
    node.tag_as('mutated')
    node.mutate() 
    
    node.tag_as('submitted')
    node.submit()

In [13]:
def cleanlog_mutate_submit(self):
    self.clean_log()
    
    self.tag_as('mutated')
    self.mutate() 
    
    self.tag_as('submitted')
    self.submit()

NodeJob.cleanlog_mutate_submit=cleanlog_mutate_submit

In [14]:
for node in root.generation(2):
    node.cleanlog_mutate_submit()

In [15]:
for node in root.generation(3):
    node.cleanlog_mutate_submit()

In [16]:
# check if all root descendants are completed 
if all([descendant.has_been('completed') for descendant in root.descendants]):
    root.tag_as('completed')

In [17]:
# retrieve the output
my_list=[]
for node in root.leaves:
    output = tree_maker.from_yaml(node.path+'/output.yaml')
    my_list.append(output['result'])

In [18]:
# sanity check
assert any(np.array(sorted(my_list))-np.array(my_list_original))==0

In [19]:
root=tree_maker.tree_from_yaml(f'/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/tree.yaml')

In [20]:
for node in root.find(filter_=
                      lambda node: node.is_leaf and 
                                   node.has_been('completed')):
    print(node.path)

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/000/000/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/000/001/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/001/000/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/001/001/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/002/000/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/002/001/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/003/000/000
/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/003/001/000


In [22]:
def my_test(node):
    output = tree_maker.from_yaml(node.path+'/output.yaml')
    return output['result']<.3


for node in root.find(filter_=lambda node: node.is_leaf and 
                                           node.has_been('completed') and 
                                           my_test(node)):
    print(node.path)
    output = tree_maker.from_yaml(node.path+'/output.yaml')
    print(f'{output["result"]}'+'\n')
    

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/000/000/000
0.22973667009077783

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/000/001/000
0.2055935932288798

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/002/000/000
0.22105104624872

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/002/001/000
0.19782074349436635

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/003/000/000
0.2646190341828162

/home/jovyan/local_host_home/CERNBox/2021/tree_maker/examples/001_example/study_000/003/001/000
0.23681016205599148

